This notebook is used to prepare amenity data (grocery stores) into distance data that can be used for clustering and visualization.



---

### Imports

In [1]:
import os
import json
import pickle
from collections import defaultdict
import gzip

import pandas as pd
import numpy as np
import scipy.spatial as spatial

import geopandas as gp
from shapely.geometry import Point, Polygon

from IPython.display import display

### Define working directory

In [2]:
ROOT = '/media/school/project/amenities'

### Read Gazetteer table

GEOIDS and lat/long for tract centers are pulled from this table.

In [3]:
gaz = pd.read_pickle(os.path.join(ROOT, '2018_5yr_cendatagov_GAZ_v3.pkl'))
gaz.GEOID = gaz.GEOID.astype(int)
gaz.columns = [x.strip() for x in gaz.columns]
gaz.head(1)

,USPS,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,AL,1001020100,9817813,28435,3.791,0.011,32.481959,-86.491338


### Read Geopandas dataframe

This is not really used for processing but gives some evidence of sanity-checks.

In [4]:
with gzip.GzipFile(os.path.join(ROOT, 'all_census_tract_shapes.json.gz'), 'r') as f:
    TRACT_ALL = json.loads(f.read().decode('utf-8'))
gpdf = gp.GeoDataFrame.from_features(TRACT_ALL['features'])
gpdf.GEOID = gpdf.GEOID.astype(int)
gpdf.head(1)

,geometry,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER
0,"POLYGON ((-93.16468 30.21663, -93.16392 30.216...",22,019,980000,1400000US22019980000,22019980000,9800,CT,5398742,2339


### Read Amenities Data

In [5]:
AMENITIES = {}
amenity_src_path = '../../amenities/source-data'

for amenity_file in os.listdir(amenity_src_path):
    print(f'Reading {amenity_file}')
    name = amenity_file.split('.')[0].strip().replace('-', '_').split('_')[0].upper()
    data = pd.read_csv(os.path.join(amenity_src_path, amenity_file))
    AMENITIES[name] = data
    
print(AMENITIES.keys(), end='\n\n')
for k, v in AMENITIES.items():
    print(f'{k} shape: {v.shape}')

Reading grocery-stores.csv
Reading gyms.csv
Reading hardware-stores.csv
Reading parks.csv
Reading medical-facilities.csv
dict_keys(['GROCERY', 'GYMS', 'HARDWARE', 'PARKS', 'MEDICAL'])

GROCERY shape: (53518, 11)
GYMS shape: (49348, 11)
HARDWARE shape: (44920, 11)
PARKS shape: (53081, 11)
MEDICAL shape: (29271, 11)


Output the amenities with the newly assigned names. These can be used for lookups in the visualization tool.

In [6]:
for name, df in AMENITIES.items():
    df.to_pickle(f'../../amenities/dataframes/{name}.pkl', protocol=4)

### Prepare the first fiew fields of the `result` table.

This will be used to aggregate results throughout the code.

In [7]:
# result with start off with the gaz GEOID and lat/long columns
result = gaz[['GEOID', 'INTPTLAT', 'INTPTLONG']]
result.head()

,GEOID,INTPTLAT,INTPTLONG
0,1001020100,32.481959,-86.491338
1,1001020200,32.475758,-86.472468
2,1001020300,32.474024,-86.459703
3,1001020400,32.471030,-86.444835
4,1001020500,32.458922,-86.421826


The following is a test to see if all geoids in gaz have corresponding geometries in gpdf.

219 in gaz are not in gpdf. These are tracts that correspond to waterways (bays, lakes, etc). A few were spot checked, for the rest, it was confirmed that ALAND was 0 in 218 records. The last record has ALAND > 0, but corresponds to tampa bay.

In [8]:
if not os.path.exists(os.path.join(ROOT, 'gaz_to_gpdf_geoid_mapping.pickle')):
    gaz_to_gpdf_geoid_mapping = {}
    for i, row in result.iterrows():
        print('\r{} '.format(i+1), end='')
        geoid = row.GEOID
        lat = float(row.INTPTLAT)
        lon = float(row.INTPTLONG)
        pt = Point(lon, lat) # order here is important!!

        # set default mapping when point not found within geometry
        gaz_to_gpdf_geoid_mapping[geoid] = None

        # if there is a matching geoid, set that
        gpdf_geoid_match = gpdf[gpdf.GEOID.isin([geoid])]
        if gpdf_geoid_match.shape[0] == 1:
            gaz_to_gpdf_geoid_mapping[geoid] = gpdf_geoid_match.GEOID.values[0]
            print('FOUND')
        # otherwise, find if it falls in a geometry
        else:
            for j, row_gpdf in gpdf.iterrows():          
                poly = row_gpdf.geometry
                found_flag = None
                if pt.within(poly):
                    print('FOUND, extra processing')
                    found_flag = True
                    gaz_to_gpdf_geoid_mapping[geoid] = row_gpdf.GEOID
                    break
            if not found_flag:
                print('===NOT FOUND===')
    with open(os.path.join(ROOT, 'gaz_to_gpdf_geoid_mapping.pickle'), 'wb') as f:
        pickle.dump(gaz_to_gpdf_geoid_mapping, f, protocol=4)
else:
    with open(os.path.join(ROOT, 'gaz_to_gpdf_geoid_mapping.pickle'), 'rb') as handle:
        gaz_to_gpdf_geoid_mapping = pickle.load(handle)    

In [9]:
len([x for x,y in gaz_to_gpdf_geoid_mapping.items() if y is not None])

72837

In [10]:
len([x for x,y in gaz_to_gpdf_geoid_mapping.items() if y is None])

219

In [11]:
gaz_geo_set = set(gaz.GEOID)
gpdf_geo_set = set(gpdf.GEOID)
print('Number of gaz geoids in gpdf: {}'.format(
    len(gaz_geo_set.intersection(gpdf_geo_set))))
print('Number of gaz geoids NOT in gpdf: {}'.format(
    gaz.shape[0] - len(gaz_geo_set.intersection(gpdf_geo_set))))

Number of gaz geoids in gpdf: 72837
Number of gaz geoids NOT in gpdf: 219


Given that these are equivalent, we can say it is likely that there are no geometries for the bays, lakes, etc that fall in the 219.

### For each business in each amenity, identify the census tract geoid in which it falls.

In [12]:
def find_within_census_tract(df):
    geoid_matches = []

    # create a dict of subsets of a merged dataframe for faster processing below
    gaz_gpdf = gaz.merge(gpdf, how='left', on='GEOID')
    gaz_gpdf_subset_dict = {}
    for state in gaz_gpdf.USPS.unique():
        gaz_gpdf_subset_dict[state] = gaz_gpdf[gaz_gpdf.USPS==state]

    for i, row in df.iterrows():
        print('\r{} '.format(i+1, end=''))
        pt = Point(row.lon_cleaned, row.lat_cleaned)
        state = row.state

        # subset to state to reduce search space
        gaz_gpdf_subset = gaz_gpdf_subset_dict[state]
        geoid_match = None # set default as None for no match
        # iterate through geometries
        for j, row_j in gaz_gpdf_subset.iterrows():
            # update geoid_match if found
            poly = row_j.geometry
            if not poly:
                continue
            if pt.within(poly):
                print('FOUND')
                geoid_match = row_j.GEOID
                break

        # append geoid_match
        geoid_matches.append(geoid_match)

        if not geoid_match:
            print('===NOT FOUND===')
    
    return geoid_matches


for name, df in AMENITIES.items():
    temp_name = f'{name}.stage1.pkl'
    temp_path = os.path.join(ROOT, temp_name)
    if os.path.exists(temp_path):
        print(f'Loading existing data at: {temp_path}')
        df = pd.read_pickle(temp_path)
        display(df.head())
        AMENITIES[name] = df
    else:
        print(f'Working on {name} amenity data. Finding GEOID in which each store falls.')
        geoid_matches = find_within_census_tract(df)
        df[f'{name}_GEOID_MATCH'] = geoid_matches
        df.to_pickle(os.path.join(ROOT, temp_name), protocol=4)        

Loading existing data at: /media/school/project/amenities/GROCERY.stage1.pkl


,zip,city,state,name,address,address_combo,postalCode,lat,lat_cleaned,lon,lon_cleaned,GROCERY_GEOID_MATCH
0,60415,Chicago Ridge,IL,ALDI,:arlem & Southwest Hwy,":arlem & Southwest Hwy,60415",60415,41.700313,41.70031,-87.797054,-87.79705,1.703182e+10
1,40219,Louisville,KY,B & E Salvage Store,.,".,40219",40219,38.128021,38.12802,-85.679443,-85.67944,2.111101e+10
2,79106,Amarillo,TX,Butch,.,".,79109",79109,35.190903,35.19090,-101.845847,-101.84585,4.837501e+10
3,60901,Kankakee,IL,Jewl,...,"...,60914",60914,41.186157,41.18616,-87.893356,-87.89336,1.709101e+10
4,54017,New Richmond,WI,Indomaret,@ Fresh market,"@ Fresh market,54017",54017,45.119296,45.11930,-92.537666,-92.53767,5.510912e+10


Loading existing data at: /media/school/project/amenities/GYMS.stage1.pkl


,zip,city,state,name,address,address_combo,postalCode,lat,lat_cleaned,lon,lon_cleaned,GYMS_GEOID_MATCH
0,30720,Dalton,GA,First Presbyterian Church Rec Room,?,"?,30720",30720,34.794614,34.794614,-84.997768,-84.997768,1.331300e+10
1,30084,Tucker,GA,The Gym,.,".,30345",30345,33.850755,33.850755,-84.247993,-84.247993,1.308902e+10
2,32826,Orlando,FL,Sick Fitness,.,".,32826",32826,28.581695,28.581695,-81.202183,-81.202183,1.209502e+10
3,49506,Grand Rapids,MI,Stairstepper,.,".,49546",49546,42.925879,42.925879,-85.605882,-85.605882,2.608100e+10
4,60104,Bellwood,IL,Boys And Girls Club,.,".,60104",60104,41.880768,41.880768,-87.882690,-87.882690,1.703182e+10


Loading existing data at: /media/school/project/amenities/HARDWARE.stage1.pkl


,zip,city,state,name,address,address_combo,postalCode,lat,lat_cleaned,lon,lon_cleaned,HARDWARE_GEOID_MATCH
0,91702,Azusa,CA,Direct Shower Door,-------,"-------,91786",91786,34.114860,34.114860,-117.657931,-117.657931,6.071001e+09
1,46777,Ossian,IN,Ace Hardware,?,"?,46777",46777,40.883478,40.883478,-85.165771,-85.165771,1.817904e+10
2,92708,Fountain Valley,CA,Work Station,???,"???,92708",92708,33.719293,33.719293,-117.962373,-117.962373,6.059099e+09
3,53916,Beaver Dam,WI,neuman pools,.,".,not_available",not_available,43.444537,43.444537,-88.859612,-88.859612,5.502796e+10
4,75229,Dallas,TX,Expediter Office,@ Home Depot,"@ Home Depot,75229",75229,32.909378,32.909378,-96.881803,-96.881803,4.811301e+10


Loading existing data at: /media/school/project/amenities/PARKS.stage1.pkl


,zip,city,state,name,address,address_combo,postalCode,lat,lat_cleaned,lon,lon_cleaned,PARKS_GEOID_MATCH
0,54201,Algoma,WI,Olson Park,-,"-,54201",54201,44.616390,44.616390,-87.444612,-87.444612,5.506196e+10
1,46932,Galveston,IN,Walton Park,-,"-,not_available",not_available,40.659611,40.659611,-86.237612,-86.237612,1.801795e+10
2,76039,Euless,TX,Chittam Park,?,"?,?",?,32.859446,32.859446,-97.078021,-97.078021,4.843911e+10
3,64014,Blue Springs,MO,Rotary Park At Railroad Lake,?,"?,not_available",not_available,39.019897,39.019897,-94.268869,-94.268869,2.909501e+10
4,52803,Davenport,IA,Tyler Park,? Grand,"? Grand,not_available",not_available,41.549636,41.549636,-90.564551,-90.564551,1.916301e+10


Loading existing data at: /media/school/project/amenities/MEDICAL.stage1.pkl


,zip,city,state,name,address,address_combo,postalCode,lat,lat_cleaned,lon,lon_cleaned,MEDICAL_GEOID_MATCH
0,21223,Baltimore,MD,Emergency Department - Saint Agnes Hospital,.,".,21223",21223,39.272825,39.272825,-76.671475,-76.671475,2.451025e+10
1,79109,Amarillo,TX,Hospital,.,".,79109",79109,35.190941,35.190941,-101.845864,-101.845864,4.837501e+10
2,81301,Durango,CO,Mercy Regional Medical Center,.,".,81301",81301,37.235922,37.235922,-107.826337,-107.826337,8.067971e+09
3,75050,Grand Prairie,TX,Children's Medical Center,....,"....,not_available",not_available,32.759488,32.759488,-96.972860,-96.972860,4.811302e+10
4,77007,Houston,TX,The Betty Ford Clinic,[Patient nam1560 Rancho Mirage,"[Patient nam1560 Rancho Mirage,not_available",not_available,29.755160,29.755160,-95.392526,-95.392526,4.820141e+10


In [13]:
for name, df in AMENITIES.items():
    print(f'\n\n{name}')
    display(df.head(3))



GROCERY


,zip,city,state,name,address,address_combo,postalCode,lat,lat_cleaned,lon,lon_cleaned,GROCERY_GEOID_MATCH
0,60415,Chicago Ridge,IL,ALDI,:arlem & Southwest Hwy,":arlem & Southwest Hwy,60415",60415,41.700313,41.70031,-87.797054,-87.79705,1.703182e+10
1,40219,Louisville,KY,B & E Salvage Store,.,".,40219",40219,38.128021,38.12802,-85.679443,-85.67944,2.111101e+10
2,79106,Amarillo,TX,Butch,.,".,79109",79109,35.190903,35.19090,-101.845847,-101.84585,4.837501e+10




GYMS


,zip,city,state,name,address,address_combo,postalCode,lat,lat_cleaned,lon,lon_cleaned,GYMS_GEOID_MATCH
0,30720,Dalton,GA,First Presbyterian Church Rec Room,?,"?,30720",30720,34.794614,34.794614,-84.997768,-84.997768,1.331300e+10
1,30084,Tucker,GA,The Gym,.,".,30345",30345,33.850755,33.850755,-84.247993,-84.247993,1.308902e+10
2,32826,Orlando,FL,Sick Fitness,.,".,32826",32826,28.581695,28.581695,-81.202183,-81.202183,1.209502e+10




HARDWARE


,zip,city,state,name,address,address_combo,postalCode,lat,lat_cleaned,lon,lon_cleaned,HARDWARE_GEOID_MATCH
0,91702,Azusa,CA,Direct Shower Door,-------,"-------,91786",91786,34.114860,34.114860,-117.657931,-117.657931,6.071001e+09
1,46777,Ossian,IN,Ace Hardware,?,"?,46777",46777,40.883478,40.883478,-85.165771,-85.165771,1.817904e+10
2,92708,Fountain Valley,CA,Work Station,???,"???,92708",92708,33.719293,33.719293,-117.962373,-117.962373,6.059099e+09




PARKS


,zip,city,state,name,address,address_combo,postalCode,lat,lat_cleaned,lon,lon_cleaned,PARKS_GEOID_MATCH
0,54201,Algoma,WI,Olson Park,-,"-,54201",54201,44.616390,44.616390,-87.444612,-87.444612,5.506196e+10
1,46932,Galveston,IN,Walton Park,-,"-,not_available",not_available,40.659611,40.659611,-86.237612,-86.237612,1.801795e+10
2,76039,Euless,TX,Chittam Park,?,"?,?",?,32.859446,32.859446,-97.078021,-97.078021,4.843911e+10




MEDICAL


,zip,city,state,name,address,address_combo,postalCode,lat,lat_cleaned,lon,lon_cleaned,MEDICAL_GEOID_MATCH
0,21223,Baltimore,MD,Emergency Department - Saint Agnes Hospital,.,".,21223",21223,39.272825,39.272825,-76.671475,-76.671475,2.451025e+10
1,79109,Amarillo,TX,Hospital,.,".,79109",79109,35.190941,35.190941,-101.845864,-101.845864,4.837501e+10
2,81301,Durango,CO,Mercy Regional Medical Center,.,".,81301",81301,37.235922,37.235922,-107.826337,-107.826337,8.067971e+09


### Find count of each amenity within each tract

In [14]:
def within_tract_counts(name, df):
    within_geoid_counts = df.groupby(f'{name}_GEOID_MATCH')[f'{name}_GEOID_MATCH'].count()
    print(within_geoid_counts.min(), within_geoid_counts.max())
    within_geoid_counts.name = f'N_{name}_WITHIN_TRACT'
    within_geoid_counts = within_geoid_counts.reset_index()
    return within_geoid_counts

AMENITY_WITHIN_TRACT_COUNTS = {}
for name, df in AMENITIES.items():
    AMENITY_WITHIN_TRACT_COUNTS[name] = within_tract_counts(name, df)
    print(f'\n\n{name}')
    display(AMENITY_WITHIN_TRACT_COUNTS[name].head(3))

1 17


GROCERY


,GROCERY_GEOID_MATCH,N_GROCERY_WITHIN_TRACT
0,1.001020e+09,1
1,1.001020e+09,3
2,1.001021e+09,1


1 26


GYMS


,GYMS_GEOID_MATCH,N_GYMS_WITHIN_TRACT
0,1.001020e+09,2
1,1.001020e+09,3
2,1.001021e+09,1


1 23


HARDWARE


,HARDWARE_GEOID_MATCH,N_HARDWARE_WITHIN_TRACT
0,1.001020e+09,2
1,1.001020e+09,1
2,1.001020e+09,2


1 27


PARKS


,PARKS_GEOID_MATCH,N_PARKS_WITHIN_TRACT
0,1.001020e+09,2
1,1.001020e+09,3
2,1.001020e+09,1


1 18


MEDICAL


,MEDICAL_GEOID_MATCH,N_MEDICAL_WITHIN_TRACT
0,1.001020e+09,2
1,1.001020e+09,4
2,1.003010e+09,2


Append the results and perform check.

In [15]:
for name, df in AMENITY_WITHIN_TRACT_COUNTS.items():
    result = result.merge(df, how='left', left_on='GEOID', right_on=f'{name}_GEOID_MATCH')
result.head()

,GEOID,INTPTLAT,INTPTLONG,GROCERY_GEOID_MATCH,N_GROCERY_WITHIN_TRACT,GYMS_GEOID_MATCH,N_GYMS_WITHIN_TRACT,HARDWARE_GEOID_MATCH,N_HARDWARE_WITHIN_TRACT,PARKS_GEOID_MATCH,N_PARKS_WITHIN_TRACT,MEDICAL_GEOID_MATCH,N_MEDICAL_WITHIN_TRACT
0,1001020100,32.481959,-86.491338,NaN,NaN,NaN,NaN,NaN,NaN,1.001020e+09,2.0,NaN,NaN
1,1001020200,32.475758,-86.472468,NaN,NaN,NaN,NaN,1.001020e+09,2.0,1.001020e+09,3.0,NaN,NaN
2,1001020300,32.474024,-86.459703,1.001020e+09,1.0,1.001020e+09,2.0,1.001020e+09,1.0,NaN,NaN,1.001020e+09,2.0
3,1001020400,32.471030,-86.444835,NaN,NaN,NaN,NaN,NaN,NaN,1.001020e+09,1.0,NaN,NaN
4,1001020500,32.458922,-86.421826,1.001020e+09,3.0,1.001020e+09,3.0,1.001020e+09,2.0,1.001020e+09,1.0,1.001020e+09,4.0


Check if number of unique geoids same in each table.

In [16]:
for name, df in AMENITY_WITHIN_TRACT_COUNTS.items():
    print(result[f'{name}_GEOID_MATCH'].nunique() ==\
          AMENITY_WITHIN_TRACT_COUNTS[name][f'{name}_GEOID_MATCH'].nunique())

True
True
True
True
True


Fill missing data with 0 (i.e. no stores found within that tract).

Drop the GEOID_MATCH field. No longer needed.

In [17]:
for col in [x for x in result.columns if x.startswith('N_')]:
    result[col] = result[col].fillna(0)
    result = result.drop(columns=['{}_GEOID_MATCH'.format(col.split('_')[1])])
result

,GEOID,INTPTLAT,INTPTLONG,N_GROCERY_WITHIN_TRACT,N_GYMS_WITHIN_TRACT,N_HARDWARE_WITHIN_TRACT,N_PARKS_WITHIN_TRACT,N_MEDICAL_WITHIN_TRACT
0,1001020100,32.481959,-86.491338,0.0,0.0,0.0,2.0,0.0
1,1001020200,32.475758,-86.472468,0.0,0.0,2.0,3.0,0.0
2,1001020300,32.474024,-86.459703,1.0,2.0,1.0,0.0,2.0
3,1001020400,32.471030,-86.444835,0.0,0.0,0.0,1.0,0.0
4,1001020500,32.458922,-86.421826,3.0,3.0,2.0,1.0,4.0
...,...,...,...,...,...,...,...,...
73051,56043000200,43.878830,-107.669052,0.0,0.0,0.0,0.0,0.0
73052,56043000301,44.014369,-107.956379,0.0,0.0,1.0,1.0,1.0
73053,56043000302,44.028771,-107.950748,0.0,1.0,0.0,0.0,0.0
73054,56045951100,43.846213,-104.570020,0.0,0.0,1.0,0.0,0.0


The following blocks of commented code were a first attempt to calculate stores that fell within a certain radius of a census tract center. This would have taken far too long to compute (~5 days).

In [18]:
### Now, compute the stores within radius of neighborhood centers
# N_GROCERY_WT_2_MI, N_GROCERY_WT_5_MI, N_GROCERY_WT_30_MI 


In [19]:
# from sklearn.metrics.pairwise import haversine_distances
# from math import radians

# # example
# bsas = [-34.83333, -58.5166646]
# paris = [49.0083899664, 2.53844117956]
# bsas_in_radians = [radians(_) for _ in bsas]
# paris_in_radians = [radians(_) for _ in paris]
# r = haversine_distances([bsas_in_radians, paris_in_radians])
# r = (r * 6371000/1000) # multiply by Earth radius to get kilometers
# r = (r * 0.621371)# km to mile
# r

In [20]:
# # Since doing many pairwise calcs, prep all items before the distance calcs are performed
# # will parallelize the distance calcs

# tract_prep = [] # store (i, prepared_location)
# # iterate through each tract
# for i, row in result.iterrows():
#     tract_lat = row.INTPTLAT
#     tract_lon = row.INTPTLONG
#     tract_in_rads = [radians(x) for x in [tract_lat, tract_lon]]
#     tract_prep.append((i, tract_in_rads))

# amen_prep = []
# for j, rowj in gro.iterrows():
#     am_lat = rowj.lat_cleaned
#     am_lon = rowj.lon_cleaned
#     am_in_rads = [radians(x) for x in [am_lat, am_lon]]
#     amen_prep.append((j, am_in_rads))
    
# print(len(tract_prep), len(amen_prep))

In [21]:
# iterator = ((x,y) for x in tract_prep for y in amen_prep) # build iterator
# distances = []
# count = 1.0
# total = len(tract_prep) * len(amen_prep)

# for a, b in iterator:
#     print('{}%'.format(round(count/total*100., 2), end=''))
#     i = a[0]
#     j = b[0]
#     r = haversine_distances([a[1], b[1]])[0][1] * 3958.754641
#     count += 1

### Calculate which stores (and number of stores) fall within some distance of the tract centers.

Create a transformer to project coordinates from latitude/longitude to a 2D plane of the USA.

In [22]:
from pyproj import Transformer
transformer = Transformer.from_crs("epsg:4326", "epsg:2163") # lat/lon to us 2d projection

Project all of the tract locations. Store for use below.

In [23]:
# transform coordinate system
tract_prep = [] # store (i, prepared_location)
# iterate through each tract
for i, row in result.iterrows():
    print('\r{} of {}'.format(i+1, result.shape[0]), end='')
    tract_lat = row.INTPTLAT
    tract_lon = row.INTPTLONG
    x, y = transformer.transform(tract_lat, tract_lon)
    point = [x, y]
    tract_prep.append((i, point))
print('tracts complete\n')

716 of 73056

1463 of 73056

2317 of 73056

3114 of 73056

3901 of 73056

...

70184 of 73056

71016 of 73056

71807 of 73056

72635 of 73056

73056 of 73056tracts complete



Project all of the amenities locations for use below.

In [24]:
def amenity_projections(df):
    amen_prep = []
    for j, rowj in df.iterrows():
        print('\r{} of {}'.format(j+1, df.shape[0]), end='')
        am_lat = rowj.lat_cleaned
        am_lon = rowj.lon_cleaned
        x, y = transformer.transform(am_lat, am_lon)
        point = [x, y]
        amen_prep.append((j, point))
    return amen_prep

AMENITIES_PROJ = {}
for name, df in AMENITIES.items():
    print(f'\n\nProjecting {name}')
    AMENITIES_PROJ[name] = amenity_projections(df)
    print(f'\n{name}: {len(AMENITIES_PROJ[name])}')
print(f'Tracts: {len(tract_prep)}')



Projecting GROCERY
53518 of 53518
GROCERY: 53518


Projecting GYMS
49348 of 49348
GYMS: 49348


Projecting HARDWARE
44920 of 44920
HARDWARE: 44920


Projecting PARKS
53081 of 53081
PARKS: 53081


Projecting MEDICAL
29271 of 29271
MEDICAL: 29271
Tracts: 73056


Build a kd-tree of the projected amenities points.

In [25]:
def amenity_kdtree(amenity_projection_list):
    points = np.array([x[1] for x in amenity_projection_list])
    print(points)
    point_tree = spatial.cKDTree(points)
    return point_tree   

AMENITY_KD_TREES = {}
for name, amenity_projection_list in AMENITIES_PROJ.items():
    print(name)
    AMENITY_KD_TREES[name] = amenity_kdtree(amenity_projection_list)

GROCERY
[[ 1008903.39597079  -291698.51000652]
 [ 1247265.06037619  -656209.377317  ]
 [ -168337.02677753 -1087554.82271786]
 ...
 [ 1686110.01810216  -300867.78168736]
 [  543273.62098321  -671832.77348739]
 [-1913137.92091447  -594763.41046583]]
GYMS
[[ 1366528.49245784 -1011749.44326778]
 [ 1451311.65010533 -1102767.86222991]
 [ 1837002.38342593 -1619951.85824966]
 ...
 [ -599261.13042061 -1452250.1295229 ]
 [-1695010.08223182 -1016198.50409325]
 [ 1227188.97791841  -339644.06235434]]
HARDWARE
[[-1618523.26252159 -1039284.93250611]
 [ 1239535.57045484  -345591.00459928]
 [-1654193.742379   -1076626.72916291]
 ...
 [ 1988543.52924897  -114085.74844058]
 [ 2006100.56953364   -32016.07705165]
 [ 2135593.54770464  -184291.66894412]]
PARKS
[[  988826.66774069    34133.2231036 ]
 [ 1155037.77020478  -385820.96458341]
 [  274398.53023507 -1342757.83485536]
 ...
 [ 1790024.13019136    52122.37377441]
 [-1448515.28850112 -1237908.33533051]
 [ 2133389.28895638   -83137.99542612]]
MEDICAL
[[ 1

Iterate through the projected tract coordinates. For each amenity, query its kdtree with these coordinates and a distance. The units produced by the projection are in meters. We specify the number of miles to the miles_to_meters function to convert it to meters.

This code stores a list of amenity indexes that fall within the radius of the census tract center. These lists are later saved to the final 'full' table.

The number in these column names specifies the number of miles that were searched (radius) from the tract centers.

In [26]:
def miles_to_meters(miles):
    meters = miles * 1609.34
    return meters

def meters_to_miles(meters):
    miles = meters / 1609.34
    return miles

def get_distance_results(tract_prep, point_tree):
    # for each tract, get a list of amenity indexes that fall within x miles 
    distance_results = defaultdict(list)
    for result_idx, locationxy in tract_prep:
        for dist in [2,5,10,25,50]: #miles
            point_list = point_tree.query_ball_point(locationxy, miles_to_meters(dist), p=np.inf)
            n_points = len(point_list)
            distance_results['dist{}'.format(dist)].append(point_list)
    return distance_results

AMENITY_DISTANCE_RESULTS = {}
for name, point_tree in AMENITY_KD_TREES.items():
    print(f'{name}')
    AMENITY_DISTANCE_RESULTS[name] = pd.DataFrame(get_distance_results(tract_prep, point_tree))
    display(AMENITY_DISTANCE_RESULTS[name].head(3))

GROCERY


,dist2,dist5,dist10,dist25,dist50
0,[],"[39075, 46917, 39758, 49391, 13067, 13782]","[39075, 46337, 49392, 46917, 39758, 49391, 130...","[49388, 31676, 22192, 35804, 49400, 21223, 216...","[49388, 17626, 7434, 32793, 9946, 6963, 9351, ..."
1,"[39075, 39758]","[39075, 39758, 49391, 13067, 13782, 19291, 886...","[39075, 46337, 49392, 46917, 39758, 49391, 130...","[49388, 31676, 22192, 35804, 49400, 21223, 216...","[49388, 42647, 17626, 7434, 32793, 9946, 6963,..."
2,"[39075, 39758, 49391]","[39075, 39758, 49391, 13067, 13782, 19291, 886...","[39075, 46337, 49392, 46917, 39758, 49391, 130...","[49388, 31676, 22192, 35804, 49400, 21223, 216...","[49388, 42647, 17626, 7434, 32793, 9946, 6963,..."


GYMS


,dist2,dist5,dist10,dist25,dist50
0,[],"[30530, 43997, 5376, 8276, 11421, 13300, 38649...","[30530, 43997, 5376, 8276, 11421, 13300, 38649...","[44001, 43994, 44002, 30918, 34752, 18336, 285...","[44001, 3785, 43994, 44002, 30918, 40466, 4399..."
1,"[30530, 5376, 8276]","[30530, 43997, 5376, 8276, 11421, 13300, 38649...","[44002, 30530, 43997, 5376, 8276, 11421, 13300...","[44001, 43994, 44002, 30918, 34752, 18336, 285...","[44001, 3785, 43994, 44002, 30918, 40466, 4399..."
2,"[30530, 5376, 8276]","[30530, 43997, 5376, 8276, 11421, 13300, 38649...","[44002, 30530, 43997, 5376, 8276, 11421, 13300...","[44001, 43994, 44002, 30918, 34752, 18336, 285...","[44001, 3785, 43994, 44002, 30918, 40466, 4399..."


HARDWARE


,dist2,dist5,dist10,dist25,dist50
0,"[10000, 6541]","[40602, 35320, 12193, 10000, 40603, 6541]","[26510, 40602, 15072, 17483, 35320, 12193, 100...","[40601, 22736, 40608, 25928, 15008, 19245, 376...","[12447, 40644, 40600, 8849, 7985, 40601, 7626,..."
1,"[35320, 10000, 6541]","[26510, 40602, 15072, 17483, 35320, 12193, 100...","[8541, 26510, 40602, 15072, 17483, 35320, 1219...","[40601, 22736, 40608, 25928, 15008, 19245, 376...","[12447, 40644, 40600, 8849, 7985, 40601, 7626,..."
2,"[35320, 10000, 6541]","[26510, 40602, 15072, 17483, 35320, 12193, 100...","[8541, 26510, 40602, 15072, 17483, 35320, 1219...","[40601, 22736, 40608, 25928, 15008, 19245, 376...","[12447, 40644, 40600, 8849, 7985, 40601, 7626,..."


PARKS


,dist2,dist5,dist10,dist25,dist50
0,"[23268, 50236, 32852, 3734, 33126, 4806, 14105]","[23268, 50236, 32852, 18134, 39212, 3734, 3312...","[23268, 39205, 50236, 32852, 7144, 18134, 3921...","[34388, 16443, 39222, 18930, 8775, 39226, 3922...","[50418, 39269, 39270, 34388, 39209, 16443, 392..."
1,"[23268, 50236, 32852, 18134, 3734, 33126, 4806...","[23268, 50236, 32852, 7144, 18134, 39212, 3734...","[8775, 23268, 50236, 32852, 7144, 18134, 39212...","[34388, 16443, 39222, 18930, 8775, 39226, 3922...","[39269, 39270, 34388, 39209, 16443, 39222, 189..."
2,"[23268, 50236, 32852, 18134, 3734, 33126, 4806...","[23268, 50236, 32852, 7144, 18134, 39212, 3734...","[8775, 23268, 50236, 32852, 7144, 18134, 39212...","[34388, 16443, 39222, 18930, 8775, 39226, 3922...","[39269, 39270, 34388, 39209, 16443, 39222, 189..."


MEDICAL


,dist2,dist5,dist10,dist25,dist50
0,[],"[6931, 2959, 25333, 6434, 6988, 6930, 25331]","[6931, 2959, 25333, 6434, 6988, 6930, 25331, 2...","[29008, 8047, 9213, 7792, 10734, 22783, 10539,...","[29008, 22321, 11707, 666, 667, 10888, 585, 94..."
1,"[2959, 25333]","[6931, 2959, 25333, 6434, 6988, 6930, 25331]","[6931, 2959, 25333, 6434, 6988, 6930, 25331, 2...","[29008, 8047, 9213, 7792, 10734, 22783, 10539,...","[29008, 22321, 11707, 666, 667, 10888, 585, 94..."
2,"[2959, 25333]","[6931, 2959, 25333, 6434, 6988, 6930, 25331, 2...","[6931, 2959, 25333, 6434, 6988, 6930, 25331, 2...","[29008, 8047, 9213, 7792, 10734, 22783, 10539,...","[29008, 22321, 11707, 666, 667, 10888, 585, 94..."


Create weighted number for amenities within x distance

Weights go as 1 / distance (radius distance, in miles)

In [27]:
for name, df in AMENITY_DISTANCE_RESULTS.items():
    for col in df.columns:
        weight = 1. / int(col[4:])
        new_col = f'wt_n_{name}_dist_{col[4:]}'.upper()
        df[new_col] =  df[col].apply(lambda x: len(x)) * weight
        AMENITY_DISTANCE_RESULTS[name] = df
    print(name)
    display(AMENITY_DISTANCE_RESULTS[name].head(3))

GROCERY


,dist2,dist5,dist10,dist25,dist50,WT_N_GROCERY_DIST_2,WT_N_GROCERY_DIST_5,WT_N_GROCERY_DIST_10,WT_N_GROCERY_DIST_25,WT_N_GROCERY_DIST_50
0,[],"[39075, 46917, 39758, 49391, 13067, 13782]","[39075, 46337, 49392, 46917, 39758, 49391, 130...","[49388, 31676, 22192, 35804, 49400, 21223, 216...","[49388, 17626, 7434, 32793, 9946, 6963, 9351, ...",0.0,1.2,1.4,2.4,2.40
1,"[39075, 39758]","[39075, 39758, 49391, 13067, 13782, 19291, 886...","[39075, 46337, 49392, 46917, 39758, 49391, 130...","[49388, 31676, 22192, 35804, 49400, 21223, 216...","[49388, 42647, 17626, 7434, 32793, 9946, 6963,...",1.0,1.8,1.5,2.4,2.44
2,"[39075, 39758, 49391]","[39075, 39758, 49391, 13067, 13782, 19291, 886...","[39075, 46337, 49392, 46917, 39758, 49391, 130...","[49388, 31676, 22192, 35804, 49400, 21223, 216...","[49388, 42647, 17626, 7434, 32793, 9946, 6963,...",1.5,2.0,1.5,2.4,2.44


GYMS


,dist2,dist5,dist10,dist25,dist50,WT_N_GYMS_DIST_2,WT_N_GYMS_DIST_5,WT_N_GYMS_DIST_10,WT_N_GYMS_DIST_25,WT_N_GYMS_DIST_50
0,[],"[30530, 43997, 5376, 8276, 11421, 13300, 38649...","[30530, 43997, 5376, 8276, 11421, 13300, 38649...","[44001, 43994, 44002, 30918, 34752, 18336, 285...","[44001, 3785, 43994, 44002, 30918, 40466, 4399...",0.0,1.6,2.0,2.8,1.88
1,"[30530, 5376, 8276]","[30530, 43997, 5376, 8276, 11421, 13300, 38649...","[44002, 30530, 43997, 5376, 8276, 11421, 13300...","[44001, 43994, 44002, 30918, 34752, 18336, 285...","[44001, 3785, 43994, 44002, 30918, 40466, 4399...",1.5,2.8,2.1,2.8,1.86
2,"[30530, 5376, 8276]","[30530, 43997, 5376, 8276, 11421, 13300, 38649...","[44002, 30530, 43997, 5376, 8276, 11421, 13300...","[44001, 43994, 44002, 30918, 34752, 18336, 285...","[44001, 3785, 43994, 44002, 30918, 40466, 4399...",1.5,3.4,2.6,2.8,1.86


HARDWARE


,dist2,dist5,dist10,dist25,dist50,WT_N_HARDWARE_DIST_2,WT_N_HARDWARE_DIST_5,WT_N_HARDWARE_DIST_10,WT_N_HARDWARE_DIST_25,WT_N_HARDWARE_DIST_50
0,"[10000, 6541]","[40602, 35320, 12193, 10000, 40603, 6541]","[26510, 40602, 15072, 17483, 35320, 12193, 100...","[40601, 22736, 40608, 25928, 15008, 19245, 376...","[12447, 40644, 40600, 8849, 7985, 40601, 7626,...",1.0,1.2,1.2,2.2,2.08
1,"[35320, 10000, 6541]","[26510, 40602, 15072, 17483, 35320, 12193, 100...","[8541, 26510, 40602, 15072, 17483, 35320, 1219...","[40601, 22736, 40608, 25928, 15008, 19245, 376...","[12447, 40644, 40600, 8849, 7985, 40601, 7626,...",1.5,1.8,1.3,2.2,2.08
2,"[35320, 10000, 6541]","[26510, 40602, 15072, 17483, 35320, 12193, 100...","[8541, 26510, 40602, 15072, 17483, 35320, 1219...","[40601, 22736, 40608, 25928, 15008, 19245, 376...","[12447, 40644, 40600, 8849, 7985, 40601, 7626,...",1.5,2.2,1.4,2.2,2.08


PARKS


,dist2,dist5,dist10,dist25,dist50,WT_N_PARKS_DIST_2,WT_N_PARKS_DIST_5,WT_N_PARKS_DIST_10,WT_N_PARKS_DIST_25,WT_N_PARKS_DIST_50
0,"[23268, 50236, 32852, 3734, 33126, 4806, 14105]","[23268, 50236, 32852, 18134, 39212, 3734, 3312...","[23268, 39205, 50236, 32852, 7144, 18134, 3921...","[34388, 16443, 39222, 18930, 8775, 39226, 3922...","[50418, 39269, 39270, 34388, 39209, 16443, 392...",3.5,2.0,1.9,2.04,1.52
1,"[23268, 50236, 32852, 18134, 3734, 33126, 4806...","[23268, 50236, 32852, 7144, 18134, 39212, 3734...","[8775, 23268, 50236, 32852, 7144, 18134, 39212...","[34388, 16443, 39222, 18930, 8775, 39226, 3922...","[39269, 39270, 34388, 39209, 16443, 39222, 189...",4.0,2.4,2.0,2.04,1.52
2,"[23268, 50236, 32852, 18134, 3734, 33126, 4806...","[23268, 50236, 32852, 7144, 18134, 39212, 3734...","[8775, 23268, 50236, 32852, 7144, 18134, 39212...","[34388, 16443, 39222, 18930, 8775, 39226, 3922...","[39269, 39270, 34388, 39209, 16443, 39222, 189...",4.0,2.6,2.1,2.04,1.52


MEDICAL


,dist2,dist5,dist10,dist25,dist50,WT_N_MEDICAL_DIST_2,WT_N_MEDICAL_DIST_5,WT_N_MEDICAL_DIST_10,WT_N_MEDICAL_DIST_25,WT_N_MEDICAL_DIST_50
0,[],"[6931, 2959, 25333, 6434, 6988, 6930, 25331]","[6931, 2959, 25333, 6434, 6988, 6930, 25331, 2...","[29008, 8047, 9213, 7792, 10734, 22783, 10539,...","[29008, 22321, 11707, 666, 667, 10888, 585, 94...",0.0,1.4,0.8,1.72,1.34
1,"[2959, 25333]","[6931, 2959, 25333, 6434, 6988, 6930, 25331]","[6931, 2959, 25333, 6434, 6988, 6930, 25331, 2...","[29008, 8047, 9213, 7792, 10734, 22783, 10539,...","[29008, 22321, 11707, 666, 667, 10888, 585, 94...",1.0,1.4,0.8,1.72,1.34
2,"[2959, 25333]","[6931, 2959, 25333, 6434, 6988, 6930, 25331, 2...","[6931, 2959, 25333, 6434, 6988, 6930, 25331, 2...","[29008, 8047, 9213, 7792, 10734, 22783, 10539,...","[29008, 22321, 11707, 666, 667, 10888, 585, 94...",1.0,1.6,0.8,1.72,1.34


Update the dist field names.    

In [28]:
for name, df in AMENITY_DISTANCE_RESULTS.items():
    dist_cols = [x for x in df.columns if x.startswith('dist')]
    for col in dist_cols:
        df.rename(columns={col: f'LIST_{name}_DIST_{col[4:]}'}, inplace=True)
    print(name)
    display(AMENITY_DISTANCE_RESULTS[name].head(3))

GROCERY


,LIST_GROCERY_DIST_2,LIST_GROCERY_DIST_5,LIST_GROCERY_DIST_10,LIST_GROCERY_DIST_25,LIST_GROCERY_DIST_50,WT_N_GROCERY_DIST_2,WT_N_GROCERY_DIST_5,WT_N_GROCERY_DIST_10,WT_N_GROCERY_DIST_25,WT_N_GROCERY_DIST_50
0,[],"[39075, 46917, 39758, 49391, 13067, 13782]","[39075, 46337, 49392, 46917, 39758, 49391, 130...","[49388, 31676, 22192, 35804, 49400, 21223, 216...","[49388, 17626, 7434, 32793, 9946, 6963, 9351, ...",0.0,1.2,1.4,2.4,2.40
1,"[39075, 39758]","[39075, 39758, 49391, 13067, 13782, 19291, 886...","[39075, 46337, 49392, 46917, 39758, 49391, 130...","[49388, 31676, 22192, 35804, 49400, 21223, 216...","[49388, 42647, 17626, 7434, 32793, 9946, 6963,...",1.0,1.8,1.5,2.4,2.44
2,"[39075, 39758, 49391]","[39075, 39758, 49391, 13067, 13782, 19291, 886...","[39075, 46337, 49392, 46917, 39758, 49391, 130...","[49388, 31676, 22192, 35804, 49400, 21223, 216...","[49388, 42647, 17626, 7434, 32793, 9946, 6963,...",1.5,2.0,1.5,2.4,2.44


GYMS


,LIST_GYMS_DIST_2,LIST_GYMS_DIST_5,LIST_GYMS_DIST_10,LIST_GYMS_DIST_25,LIST_GYMS_DIST_50,WT_N_GYMS_DIST_2,WT_N_GYMS_DIST_5,WT_N_GYMS_DIST_10,WT_N_GYMS_DIST_25,WT_N_GYMS_DIST_50
0,[],"[30530, 43997, 5376, 8276, 11421, 13300, 38649...","[30530, 43997, 5376, 8276, 11421, 13300, 38649...","[44001, 43994, 44002, 30918, 34752, 18336, 285...","[44001, 3785, 43994, 44002, 30918, 40466, 4399...",0.0,1.6,2.0,2.8,1.88
1,"[30530, 5376, 8276]","[30530, 43997, 5376, 8276, 11421, 13300, 38649...","[44002, 30530, 43997, 5376, 8276, 11421, 13300...","[44001, 43994, 44002, 30918, 34752, 18336, 285...","[44001, 3785, 43994, 44002, 30918, 40466, 4399...",1.5,2.8,2.1,2.8,1.86
2,"[30530, 5376, 8276]","[30530, 43997, 5376, 8276, 11421, 13300, 38649...","[44002, 30530, 43997, 5376, 8276, 11421, 13300...","[44001, 43994, 44002, 30918, 34752, 18336, 285...","[44001, 3785, 43994, 44002, 30918, 40466, 4399...",1.5,3.4,2.6,2.8,1.86


HARDWARE


,LIST_HARDWARE_DIST_2,LIST_HARDWARE_DIST_5,LIST_HARDWARE_DIST_10,LIST_HARDWARE_DIST_25,LIST_HARDWARE_DIST_50,WT_N_HARDWARE_DIST_2,WT_N_HARDWARE_DIST_5,WT_N_HARDWARE_DIST_10,WT_N_HARDWARE_DIST_25,WT_N_HARDWARE_DIST_50
0,"[10000, 6541]","[40602, 35320, 12193, 10000, 40603, 6541]","[26510, 40602, 15072, 17483, 35320, 12193, 100...","[40601, 22736, 40608, 25928, 15008, 19245, 376...","[12447, 40644, 40600, 8849, 7985, 40601, 7626,...",1.0,1.2,1.2,2.2,2.08
1,"[35320, 10000, 6541]","[26510, 40602, 15072, 17483, 35320, 12193, 100...","[8541, 26510, 40602, 15072, 17483, 35320, 1219...","[40601, 22736, 40608, 25928, 15008, 19245, 376...","[12447, 40644, 40600, 8849, 7985, 40601, 7626,...",1.5,1.8,1.3,2.2,2.08
2,"[35320, 10000, 6541]","[26510, 40602, 15072, 17483, 35320, 12193, 100...","[8541, 26510, 40602, 15072, 17483, 35320, 1219...","[40601, 22736, 40608, 25928, 15008, 19245, 376...","[12447, 40644, 40600, 8849, 7985, 40601, 7626,...",1.5,2.2,1.4,2.2,2.08


PARKS


,LIST_PARKS_DIST_2,LIST_PARKS_DIST_5,LIST_PARKS_DIST_10,LIST_PARKS_DIST_25,LIST_PARKS_DIST_50,WT_N_PARKS_DIST_2,WT_N_PARKS_DIST_5,WT_N_PARKS_DIST_10,WT_N_PARKS_DIST_25,WT_N_PARKS_DIST_50
0,"[23268, 50236, 32852, 3734, 33126, 4806, 14105]","[23268, 50236, 32852, 18134, 39212, 3734, 3312...","[23268, 39205, 50236, 32852, 7144, 18134, 3921...","[34388, 16443, 39222, 18930, 8775, 39226, 3922...","[50418, 39269, 39270, 34388, 39209, 16443, 392...",3.5,2.0,1.9,2.04,1.52
1,"[23268, 50236, 32852, 18134, 3734, 33126, 4806...","[23268, 50236, 32852, 7144, 18134, 39212, 3734...","[8775, 23268, 50236, 32852, 7144, 18134, 39212...","[34388, 16443, 39222, 18930, 8775, 39226, 3922...","[39269, 39270, 34388, 39209, 16443, 39222, 189...",4.0,2.4,2.0,2.04,1.52
2,"[23268, 50236, 32852, 18134, 3734, 33126, 4806...","[23268, 50236, 32852, 7144, 18134, 39212, 3734...","[8775, 23268, 50236, 32852, 7144, 18134, 39212...","[34388, 16443, 39222, 18930, 8775, 39226, 3922...","[39269, 39270, 34388, 39209, 16443, 39222, 189...",4.0,2.6,2.1,2.04,1.52


MEDICAL


,LIST_MEDICAL_DIST_2,LIST_MEDICAL_DIST_5,LIST_MEDICAL_DIST_10,LIST_MEDICAL_DIST_25,LIST_MEDICAL_DIST_50,WT_N_MEDICAL_DIST_2,WT_N_MEDICAL_DIST_5,WT_N_MEDICAL_DIST_10,WT_N_MEDICAL_DIST_25,WT_N_MEDICAL_DIST_50
0,[],"[6931, 2959, 25333, 6434, 6988, 6930, 25331]","[6931, 2959, 25333, 6434, 6988, 6930, 25331, 2...","[29008, 8047, 9213, 7792, 10734, 22783, 10539,...","[29008, 22321, 11707, 666, 667, 10888, 585, 94...",0.0,1.4,0.8,1.72,1.34
1,"[2959, 25333]","[6931, 2959, 25333, 6434, 6988, 6930, 25331]","[6931, 2959, 25333, 6434, 6988, 6930, 25331, 2...","[29008, 8047, 9213, 7792, 10734, 22783, 10539,...","[29008, 22321, 11707, 666, 667, 10888, 585, 94...",1.0,1.4,0.8,1.72,1.34
2,"[2959, 25333]","[6931, 2959, 25333, 6434, 6988, 6930, 25331, 2...","[6931, 2959, 25333, 6434, 6988, 6930, 25331, 2...","[29008, 8047, 9213, 7792, 10734, 22783, 10539,...","[29008, 22321, 11707, 666, 667, 10888, 585, 94...",1.0,1.6,0.8,1.72,1.34


Append the newly calc'd info to the `result` table

In [29]:
for name, df in AMENITY_DISTANCE_RESULTS.items():
    result = pd.concat([result, df], axis=1)
result.head(3)

,GEOID,INTPTLAT,INTPTLONG,N_GROCERY_WITHIN_TRACT,N_GYMS_WITHIN_TRACT,N_HARDWARE_WITHIN_TRACT,N_PARKS_WITHIN_TRACT,N_MEDICAL_WITHIN_TRACT,LIST_GROCERY_DIST_2,LIST_GROCERY_DIST_5,...,LIST_MEDICAL_DIST_2,LIST_MEDICAL_DIST_5,LIST_MEDICAL_DIST_10,LIST_MEDICAL_DIST_25,LIST_MEDICAL_DIST_50,WT_N_MEDICAL_DIST_2,WT_N_MEDICAL_DIST_5,WT_N_MEDICAL_DIST_10,WT_N_MEDICAL_DIST_25,WT_N_MEDICAL_DIST_50
0,1001020100,32.481959,-86.491338,0.0,0.0,0.0,2.0,0.0,[],"[39075, 46917, 39758, 49391, 13067, 13782]",...,[],"[6931, 2959, 25333, 6434, 6988, 6930, 25331]","[6931, 2959, 25333, 6434, 6988, 6930, 25331, 2...","[29008, 8047, 9213, 7792, 10734, 22783, 10539,...","[29008, 22321, 11707, 666, 667, 10888, 585, 94...",0.0,1.4,0.8,1.72,1.34
1,1001020200,32.475758,-86.472468,0.0,0.0,2.0,3.0,0.0,"[39075, 39758]","[39075, 39758, 49391, 13067, 13782, 19291, 886...",...,"[2959, 25333]","[6931, 2959, 25333, 6434, 6988, 6930, 25331]","[6931, 2959, 25333, 6434, 6988, 6930, 25331, 2...","[29008, 8047, 9213, 7792, 10734, 22783, 10539,...","[29008, 22321, 11707, 666, 667, 10888, 585, 94...",1.0,1.4,0.8,1.72,1.34
2,1001020300,32.474024,-86.459703,1.0,2.0,1.0,0.0,2.0,"[39075, 39758, 49391]","[39075, 39758, 49391, 13067, 13782, 19291, 886...",...,"[2959, 25333]","[6931, 2959, 25333, 6434, 6988, 6930, 25331, 2...","[6931, 2959, 25333, 6434, 6988, 6930, 25331, 2...","[29008, 8047, 9213, 7792, 10734, 22783, 10539,...","[29008, 22321, 11707, 666, 667, 10888, 585, 94...",1.0,1.6,0.8,1.72,1.34


Output this full table to a file. This is rather large. It could be used for the visualization but is not needed for clustering.

In [30]:
OUTPATH_ROOT = '../../amenities'
FULL_NAME = 'amenities_full.pkl.gz'

result.to_pickle(os.path.join(OUTPATH_ROOT, FULL_NAME), protocol=4, compression='gzip')

Output a much smaller version (per disk storage) of the table without the list columns. This would be ideal to use for clustering.

In [31]:
FEAT_NAME = 'amenities_features.pkl'
result[[x for x in result.columns if not x.startswith('LIST_')]].to_pickle(
    os.path.join(OUTPATH_ROOT, FEAT_NAME), protocol=4
)

Output a table with just GEOID + the list columns ending with radius 10. This is a slimmed-down table to reduce memory requirements on the vis app.

In [32]:
RADIUS_25_NAME = 'amenities_25mi_for_vis.pkl.gz'
columns = ['GEOID'] + [x for x in result.columns if x.startswith('LIST_') and x.endswith('25')]
result[columns].to_pickle(os.path.join(OUTPATH_ROOT, RADIUS_25_NAME), protocol=4, compression='gzip')

Test read each output

In [33]:
pd.set_option('display.max_columns', None)

In [34]:
print(FULL_NAME)
display(pd.read_pickle(os.path.join(OUTPATH_ROOT, FULL_NAME), compression='gzip').head())
print(FEAT_NAME)
display(pd.read_pickle(os.path.join(OUTPATH_ROOT, FEAT_NAME)).head())
print(RADIUS_25_NAME)
display(pd.read_pickle(os.path.join(OUTPATH_ROOT, RADIUS_25_NAME), compression='gzip').head())

amenities_full.pkl.gz


,GEOID,INTPTLAT,INTPTLONG,N_GROCERY_WITHIN_TRACT,N_GYMS_WITHIN_TRACT,N_HARDWARE_WITHIN_TRACT,N_PARKS_WITHIN_TRACT,N_MEDICAL_WITHIN_TRACT,LIST_GROCERY_DIST_2,LIST_GROCERY_DIST_5,LIST_GROCERY_DIST_10,LIST_GROCERY_DIST_25,LIST_GROCERY_DIST_50,WT_N_GROCERY_DIST_2,WT_N_GROCERY_DIST_5,WT_N_GROCERY_DIST_10,WT_N_GROCERY_DIST_25,WT_N_GROCERY_DIST_50,LIST_GYMS_DIST_2,LIST_GYMS_DIST_5,LIST_GYMS_DIST_10,LIST_GYMS_DIST_25,LIST_GYMS_DIST_50,WT_N_GYMS_DIST_2,WT_N_GYMS_DIST_5,WT_N_GYMS_DIST_10,WT_N_GYMS_DIST_25,WT_N_GYMS_DIST_50,LIST_HARDWARE_DIST_2,LIST_HARDWARE_DIST_5,LIST_HARDWARE_DIST_10,LIST_HARDWARE_DIST_25,LIST_HARDWARE_DIST_50,WT_N_HARDWARE_DIST_2,WT_N_HARDWARE_DIST_5,WT_N_HARDWARE_DIST_10,WT_N_HARDWARE_DIST_25,WT_N_HARDWARE_DIST_50,LIST_PARKS_DIST_2,LIST_PARKS_DIST_5,LIST_PARKS_DIST_10,LIST_PARKS_DIST_25,LIST_PARKS_DIST_50,WT_N_PARKS_DIST_2,WT_N_PARKS_DIST_5,WT_N_PARKS_DIST_10,WT_N_PARKS_DIST_25,WT_N_PARKS_DIST_50,LIST_MEDICAL_DIST_2,LIST_MEDICAL_DIST_5,LIST_MEDICAL_DIST_10,LIST_MEDICAL_DIST_25,LIST_MEDICAL_DIST_50,WT_N_MEDICAL_DIST_2,WT_N_MEDICAL_DIST_5,WT_N_MEDICAL_DIST_10,WT_N_MEDICAL_DIST_25,WT_N_MEDICAL_DIST_50
0,1001020100,32.481959,-86.491338,0.0,0.0,0.0,2.0,0.0,[],"[39075, 46917, 39758, 49391, 13067, 13782]","[39075, 46337, 49392, 46917, 39758, 49391, 130...","[49388, 31676, 22192, 35804, 49400, 21223, 216...","[49388, 17626, 7434, 32793, 9946, 6963, 9351, ...",0.0,1.2,1.4,2.4,2.40,[],"[30530, 43997, 5376, 8276, 11421, 13300, 38649...","[30530, 43997, 5376, 8276, 11421, 13300, 38649...","[44001, 43994, 44002, 30918, 34752, 18336, 285...","[44001, 3785, 43994, 44002, 30918, 40466, 4399...",0.0,1.6,2.0,2.8,1.88,"[10000, 6541]","[40602, 35320, 12193, 10000, 40603, 6541]","[26510, 40602, 15072, 17483, 35320, 12193, 100...","[40601, 22736, 40608, 25928, 15008, 19245, 376...","[12447, 40644, 40600, 8849, 7985, 40601, 7626,...",1.0,1.2,1.2,2.20,2.08,"[23268, 50236, 32852, 3734, 33126, 4806, 14105]","[23268, 50236, 32852, 18134, 39212, 3734, 3312...","[23268, 39205, 50236, 32852, 7144, 18134, 3921...","[34388, 16443, 39222, 18930, 8775, 39226, 3922...","[50418, 39269, 39270, 34388, 39209, 16443, 392...",3.5,2.0,1.9,2.04,1.52,[],"[6931, 2959, 25333, 6434, 6988, 6930, 25331]","[6931, 2959, 25333, 6434, 6988, 6930, 25331, 2...","[29008, 8047, 9213, 7792, 10734, 22783, 10539,...","[29008, 22321, 11707, 666, 667, 10888, 585, 94...",0.0,1.4,0.8,1.72,1.34
1,1001020200,32.475758,-86.472468,0.0,0.0,2.0,3.0,0.0,"[39075, 39758]","[39075, 39758, 49391, 13067, 13782, 19291, 886...","[39075, 46337, 49392, 46917, 39758, 49391, 130...","[49388, 31676, 22192, 35804, 49400, 21223, 216...","[49388, 42647, 17626, 7434, 32793, 9946, 6963,...",1.0,1.8,1.5,2.4,2.44,"[30530, 5376, 8276]","[30530, 43997, 5376, 8276, 11421, 13300, 38649...","[44002, 30530, 43997, 5376, 8276, 11421, 13300...","[44001, 43994, 44002, 30918, 34752, 18336, 285...","[44001, 3785, 43994, 44002, 30918, 40466, 4399...",1.5,2.8,2.1,2.8,1.86,"[35320, 10000, 6541]","[26510, 40602, 15072, 17483, 35320, 12193, 100...","[8541, 26510, 40602, 15072, 17483, 35320, 1219...","[40601, 22736, 40608, 25928, 15008, 19245, 376...","[12447, 40644, 40600, 8849, 7985, 40601, 7626,...",1.5,1.8,1.3,2.20,2.08,"[23268, 50236, 32852, 18134, 3734, 33126, 4806...","[23268, 50236, 32852, 7144, 18134, 39212, 3734...","[8775, 23268, 50236, 32852, 7144, 18134, 39212...","[34388, 16443, 39222, 18930, 8775, 39226, 3922...","[39269, 39270, 34388, 39209, 16443, 39222, 189...",4.0,2.4,2.0,2.04,1.52,"[2959, 25333]","[6931, 2959, 25333, 6434, 6988, 6930, 25331]","[6931, 2959, 25333, 6434, 6988, 6930, 25331, 2...","[29008, 8047, 9213, 7792, 10734, 22783, 10539,...","[29008, 22321, 11707, 666, 667, 10888, 585, 94...",1.0,1.4,0.8,1.72,1.34
2,1001020300,32.474024,-86.459703,1.0,2.0,1.0,0.0,2.0,"[39075, 39758, 49391]","[39075, 39758, 49391, 13067, 13782, 19291, 886...","[39075, 46337, 49392, 46917, 39758, 49391, 130...","[49388, 31676, 22192, 35804, 49400, 21223, 216...","[49388, 42647, 17626, 7434, 32793, 9946, 6963,...",1.5

amenities_features.pkl


,GEOID,INTPTLAT,INTPTLONG,N_GROCERY_WITHIN_TRACT,N_GYMS_WITHIN_TRACT,N_HARDWARE_WITHIN_TRACT,N_PARKS_WITHIN_TRACT,N_MEDICAL_WITHIN_TRACT,WT_N_GROCERY_DIST_2,WT_N_GROCERY_DIST_5,WT_N_GROCERY_DIST_10,WT_N_GROCERY_DIST_25,WT_N_GROCERY_DIST_50,WT_N_GYMS_DIST_2,WT_N_GYMS_DIST_5,WT_N_GYMS_DIST_10,WT_N_GYMS_DIST_25,WT_N_GYMS_DIST_50,WT_N_HARDWARE_DIST_2,WT_N_HARDWARE_DIST_5,WT_N_HARDWARE_DIST_10,WT_N_HARDWARE_DIST_25,WT_N_HARDWARE_DIST_50,WT_N_PARKS_DIST_2,WT_N_PARKS_DIST_5,WT_N_PARKS_DIST_10,WT_N_PARKS_DIST_25,WT_N_PARKS_DIST_50,WT_N_MEDICAL_DIST_2,WT_N_MEDICAL_DIST_5,WT_N_MEDICAL_DIST_10,WT_N_MEDICAL_DIST_25,WT_N_MEDICAL_DIST_50
0,1001020100,32.481959,-86.491338,0.0,0.0,0.0,2.0,0.0,0.0,1.2,1.4,2.4,2.40,0.0,1.6,2.0,2.8,1.88,1.0,1.2,1.2,2.20,2.08,3.5,2.0,1.9,2.04,1.52,0.0,1.4,0.8,1.72,1.34
1,1001020200,32.475758,-86.472468,0.0,0.0,2.0,3.0,0.0,1.0,1.8,1.5,2.4,2.44,1.5,2.8,2.1,2.8,1.86,1.5,1.8,1.3,2.20,2.08,4.0,2.4,2.0,2.04,1.52,1.0,1.4,0.8,1.72,1.34
2,1001020300,32.474024,-86.459703,1.0,2.0,1.0,0.0,2.0,1.5,2.0,1.5,2.4,2.44,1.5,3.4,2.6,2.8,1.86,1.5,2.2,1.4,2.20,2.08,4.0,2.6,2.1,2.04,1.52,1.0,1.6,0.8,1.72,1.34
3,1001020400,32.471030,-86.444835,0.0,0.0,0.0,1.0,0.0,2.5,2.0,1.6,2.4,2.44,3.5,3.4,2.8,2.8,1.86,2.5,2.4,1.7,2.20,2.08,3.0,3.0,2.5,2.04,1.50,3.0,1.6,0.8,1.72,1.32
4,1001020500,32.458922,-86.421826,3.0,3.0,2.0,1.0,4.0,3.0,2.0,2.3,2.4,2.40,4.5,3.8,3.0,2.8,1.82,2.0,2.4,1.9,2.16,2.06,2.0,3.0,3.0,2.08,1.52,3.0,1.6,1.8,1.68,1.28


amenities_25mi_for_vis.pkl.gz


,GEOID,LIST_GROCERY_DIST_25,LIST_GYMS_DIST_25,LIST_HARDWARE_DIST_25,LIST_PARKS_DIST_25,LIST_MEDICAL_DIST_25
0,1001020100,"[49388, 31676, 22192, 35804, 49400, 21223, 216...","[44001, 43994, 44002, 30918, 34752, 18336, 285...","[40601, 22736, 40608, 25928, 15008, 19245, 376...","[34388, 16443, 39222, 18930, 8775, 39226, 3922...","[29008, 8047, 9213, 7792, 10734, 22783, 10539,..."
1,1001020200,"[49388, 31676, 22192, 35804, 49400, 21223, 216...","[44001, 43994, 44002, 30918, 34752, 18336, 285...","[40601, 22736, 40608, 25928, 15008, 19245, 376...","[34388, 16443, 39222, 18930, 8775, 39226, 3922...","[29008, 8047, 9213, 7792, 10734, 22783, 10539,..."
2,1001020300,"[49388, 31676, 22192, 35804, 49400, 21223, 216...","[44001, 43994, 44002, 30918, 34752, 18336, 285...","[40601, 22736, 40608, 25928, 15008, 19245, 376...","[34388, 16443, 39222, 18930, 8775, 39226, 3922...","[29008, 8047, 9213, 7792, 10734, 22783, 10539,..."
3,1001020400,"[49388, 31676, 22192, 35804, 49400, 21223, 216...","[44001, 43994, 44002, 30918, 34752, 18336, 285...","[40601, 22736, 40608, 25928, 15008, 19245, 376...","[34388, 16443, 39222, 18930, 8775, 39226, 3922...","[29008, 8047, 9213, 7792, 10734, 22783, 10539,..."
4,1001020500,"[49388, 31676, 22192, 35804, 49400, 21223, 216...","[44001, 43994, 44002, 30918, 34752, 18336, 285...","[40601, 22736, 40608, 25928, 15008, 19245, 376...","[34388, 16443, 39222, 18930, 8775, 39226, 3922...","[29008, 8047, 9213, 7792, 10734, 22783, 10539,..."
